In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
# from langchain_chroma import Chroma
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

# 1. 문서 로드

In [3]:
def pdf_loader(file_path_list):
    result = []
    for i, file_path in enumerate(file_path_list):
        print(f"{i + 1}/{len(file_path_list)} 문서 로드중")
        loader = PyPDFLoader(file_path)
        docs = loader.load()
        print(f"총 {len(docs)}장의 문서 로드")
        result.extend(docs)

    return result

In [4]:
file_path_list = ['../data/ESG_2023_tot.pdf', '../data/ESG_2024_fact_book.pdf']

docs = pdf_loader(file_path_list)

1/2 문서 로드중
총 49장의 문서 로드
2/2 문서 로드중
총 44장의 문서 로드


In [5]:
docs[49]

Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 20.3 (Macintosh)', 'creationdate': '2025-07-23T08:13:09+07:00', 'moddate': '2025-07-23T08:13:22+07:00', 'trapped': '/False', 'source': '../data/ESG_2024_fact_book.pdf', 'total_pages': 44, 'page': 0, 'page_label': '1'}, page_content='')

# 2. 텍스트 스플리터(청킹)

In [6]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", " ", ""]
)

chunks = splitter.split_documents(docs)
print(len(chunks), "전체 잘린 chunk 사이즈")

119 전체 잘린 chunk 사이즈


In [7]:
print(chunks[10].page_content)

16.8%
철강 · 비철금속
8.8%
석유 · 화학
8.8%
자동차
7 .2%
기타산업
7 .2%
섬유 • 의류
5.6%
건설
5.6%
음식료
5.6%
전자기기
4.8%
기계 · 정밀
4.8%
목재 · 종이
4.8%
반도체 
전자부품
3.2%
비금속광물
3.2%
운수 · 보관
3.2%
정보 · 방송통신
2.4%
폐기물처리 · 재활용
8.0%
기타(조선, 유통 유틸리티 등) 업종별 
컨설팅 현황
녹색금융
('23년 12월 기준)
140회 40회
»
»
»
- 9 -


# 3. 임베딩 생성 chromadb 저장

In [ ]:
db_path = "../07_vectorstore/chromadb_rag_sinhanESG"

embedding = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding,
    persist_directory=db_path,
    collection_name="samsung_2024"
)
print("벡터 저장소 저장 완료")

벡터 저장소 저장 완료


# 4. 검색기 구성 (retriever)

In [9]:
retriever = vectorstore.as_retriever()

# 5. 기본 체인 만들기

In [16]:
# 1. 프롬프트
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """
     주어진 컨텍스트만 근거로 간결하고 정확하게 대답해라
     컨텍스트에 없으면 문서에 "근거 없음" 이라고 말해라
     
     [출처 작성 규칙]
     - 대답에는 어떤 문서의 몇페이지를 근거하고 있는지 리스트로 작성하라

     [컨텍스트]
     {context}
     """),
    ("user", "{question}"),
])

# 2. 모델 선택
model = ChatOpenAI(
    temperature=0,
    model = "gpt-4.1-mini",
    verbose=True
)

# 3. output parser 선택
outputparser = StrOutputParser()

# 4. chain 생성
chain = rag_prompt | model | outputparser


# 6. RAG 체인 만들기

In [17]:
# 문서를 합치는 함수
def format_docs(docs):
    return "\n\---\n\n".join([f"메타데이터: {doc.metadata}, 컨텐츠: " + doc.page_content for doc in docs])

In [18]:
rag_chain = (
    {
        "context" : retriever | RunnableLambda(format_docs),
        "question" : RunnablePassthrough()
    }
    | chain
)
rag_chain

{
  context: VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000020BB2D3E7D0>, search_kwargs={})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n     주어진 컨텍스트만 근거로 간결하고 정확하게 대답해라\n     컨텍스트에 없으면 문서에 "근거 없음" 이라고 말해라\n     \n     [출처 작성 규칙]\n     - 대답에는 어떤 문서의 몇페이지를 근거하고 있는지 리스트로 작성하라\n\n     [컨텍스트]\n     {context}\n     '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000020BB6596980>, async_client=<openai.reso

In [20]:
result = rag_chain.invoke("신한은행에서 진행하는 esg 컨설팅 중 온실가스와 관련된 사업은 어떤것이 있나요?")
print(result)

신한은행의 ESG 컨설팅 중 온실가스와 관련된 사업은 다음과 같습니다:

- 온실가스 배출량 산정 및 공개
- 온실가스 감축을 위한 이행 지원 (금융지원 연계 등)
- 친환경 경영 가이드 제공 및 에너지 효율화 전환금융 솔루션 지원
- CEMP사업(기업의 건물 에너지 효율화 사회공헌 사업과 외부사업을 연결하여 온실가스 감축과 에너지 복지를 구현하는 사업) 연계

이 외에도 중소·중견기업을 대상으로 ESG경영 진단 및 적용 컨설팅과 녹색금융 연계 솔루션을 제공하여 온실가스 감축을 지원하고 있습니다.

[출처: ESG_2023_tot.pdf, 9~10페이지, 19페이지]


In [21]:
result = rag_chain.invoke("이 보고서에서 2024년 신한은행 ESG 전략은?")
print(result)

2024년 신한은행 ESG 전략은 'L.E.A.D ESG' 전략으로, 신한금융그룹 2025 ESG 중기전략과 신한의 핵심가치에 기반하여 수립되었습니다. 주요 내용은 다음과 같습니다:

- Lead ESG Standard: ESG 이니셔티브 선도 및 규제 대응 강화  
- Expand ESG Alliance: 신사업 확장 및 ESG 금융 생태계 조성  
- Accelerate ESG Finance: 친환경 및 사회적 금융 증대  
- Design ESG Brand-Identity: ESG 내재화 및 브랜딩 강화  

이 전략은 탄소중립 가속화, 친환경 금융 리딩, 스타트업 및 청년 지원, 사회공헌 및 금융소비자 보호, 공시 및 사업 투명성 제고, 다양성 및 포용성 확대 등을 포함합니다.

또한, ESG 경영을 위한 이사회 및 산하 위원회(ESG 위원회, ESG 경영위원회, ESG 운영위원회)를 통해 전략 수립과 실행, 의사결정 체계를 운영하고 있습니다.

[근거 문서]  
- ESG_2023_tot.pdf, 42페이지  
- ESG_2024_fact_book.pdf, 3페이지  
- ESG_2024_fact_book.pdf, 4페이지
